# **Tugas Besar 2**
## Kelompok 8:
1. 16520289&emsp;Gagas Praharsa Bahar
2. 16520299&emsp;Malik Akbar Hashemi Rafsanjani
3. 16520309&emsp;Alifia Rahmah
4. 16520319&emsp;Ng Kyle

## Sumber data: 
Trending YouTube Video Statistics (US Videos) - Mitchell J ([Kaggle](https://www.kaggle.com/datasnaek/youtube-new))

### Data utama : USvideos.csv (59.85 MB)
Sumber data dengan ukuran 40949 baris x 16 kolom

### Data penunjang : US_category_id.json (8.3 KB)
referensi category_id dengan category_name yang akan membantu pada pengolahan data selanjutnya.


## **Inisialisasi**

In [ ]:
# import module yang dibutuhkan
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from pandas.plotting import scatter_matrix

# link file csv dan json
url_csv = 'https://github.com/malikrafsan/test/raw/main/USvideos.csv'
url_json = 'https://github.com/malikrafsan/Tugas-Besar2-KU1102/raw/main/US_category_id.json'

In [ ]:
# Membaca file csv dan json
df = pd.read_csv(url_csv)
data = pd.read_json(url_json)

In [ ]:
# Membuat list berisi tuple pasangan category_id dan category_name
lst = []
for i in range(len(data["items"])):
    lst.append((data["items"][i]["id"],data["items"][i]["snippet"]["title"]))

In [ ]:
# Menambahkan kolom "category_name" dan mengisinya sesuai category_id
df.insert(len(df.columns),'category_name','')
for i in range(len(df)):
    for j in range(len(lst)):
        if int(df["category_id"][i]) == int(lst[j][0]):
            df["category_name"][i] = lst[j][1]

In [ ]:
df.head()

## **Karakteristik data**

In [ ]:
#Ukuran Data
print("Banyak Baris :", len(df))
print("Banyak Kolom :", len(df.columns))

In [ ]:
#Mencari data unik
for i in range (len(df.columns)):
  print("Data unik kolom", df.columns[i],"ada sebanyak", len(df.iloc[:,i].unique()))

In [ ]:
#Mencari range atau member category tiap kolom time series dan numerikal
num_date = ["trending_date","publish_time","views","likes","dislikes"]
for i in num_date:
  print("Kolom", i, "memiliki range :", df[i].min(), "--",df[i].max())
print()

In [ ]:
#Mencari nilai unik kategorikal
kategorikal = ["category_id", "category_name"]
for i in kategorikal:
  print("Member pada kolom", i, "adalah:")
  for j in df[i].unique():
    print(j)
  print()

In [ ]:
df.info()

# **Data Preprocessing**

In [ ]:
# delete kolom yang tidak diperlukan
df.drop('video_id', inplace=True, axis=1)       # tidak dibutuhkan
df.drop('thumbnail_link', inplace=True, axis=1) # tidak dibutuhkan
df.drop('description', inplace=True, axis=1)    # tidak dibutuhkan
df.drop('category_id', inplace=True, axis=1)    # digantikan category_name
df.drop('tags', inplace=True, axis=1)           # tidak dibutuhkan

In [ ]:
# mengubah tipe data publish_time dan trending_date menjadi time series 
df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
df["trending_date"]=pd.to_datetime(df["trending_date"],format="%y.%d.%m")

In [ ]:
df.head()

In [ ]:
df.info()

# **Data Analysis**

### Statistik umum setiap atribut data numerik

In [ ]:
# informasi singkat data numerik
df.describe()

In [ ]:
# Rata-rata
df.mean()

In [ ]:
# Standar deviasi
df.std()

In [ ]:
#Ekstremum minimum
df.min(numeric_only=True)

In [ ]:
#Ekstremum maksimum
df.max(numeric_only=True)

In [ ]:
#persentil
persentil = [0.1,0.25,0.5,0.75,0.9]
for i in persentil:
  print("Persentil", i*100,"%")
  print(df.quantile(i))
  print()

### Statistik data Channel

In [ ]:
# tabel frekuensi daftar 10 channel yang pernah masuk trending (paling sering)
df['channel_title'].value_counts()[:10]

### Statistik data Category

In [ ]:
# statistik yang dikelompokkan berdasarkan category_name
frekuensi_kategori = df.groupby("category_name").sum()
frekuensi_kategori

In [ ]:
# tabel frekuensi category-category yang pernah masuk trending 
# dan seberapa sering video dengan kategori tersebut masuk
df['category_name'].value_counts()

### Statistik data Views

In [ ]:
# banyak views terbanyak dan tersedikit
print("View terbanyak:", df["views"].max())
print("View tersedikit:", df["views"].min())

In [ ]:
# Video dengan banyak views terbesar
df.loc[df['views'] == df['views'].max()]

In [ ]:
# Video dengan banyak views terkecil
df.loc[df['views'] == df['views'].min()]['title']

### Statistik data Likes

In [ ]:
# banyak likes terbanyak dan tersedikit
print("Like terbanyak:", df["likes"].max())
print("Like tersedikit:", df["likes"].min())

In [ ]:
# video dengan banyak likes terbesar
df.loc[df['likes'] == df['likes'].max()]

In [ ]:
# video dengan banyak likes terkecil
df.loc[df['likes'] == df['likes'].min()]

### Statistik data Dislikes

In [ ]:
# banyak dislike terbanyak dan tersedikit
print("Dislike terbanyak:", df["dislikes"].max())
print("Dislike tersedikit:", df["dislikes"].min())

In [ ]:
# video dengan dislike terbanyak
df.loc[df['dislikes'] == df['dislikes'].max()]

In [ ]:
# video dengan dislike tersedikit
df.loc[df['dislikes'] == df['dislikes'].min()]

### Statistik data Comment counts

In [ ]:
# banyak komentar terbanyak dan tersedikit
print("Comment counts terbanyak:", df["comment_count"].max())
print("Comment counts tersedikit:", df["comment_count"].min())

In [ ]:
# video dengan komentar terbanyak
df.loc[df['comment_count'] == df['comment_count'].max()]

In [ ]:
# video dengan komentar tersedikit
df.loc[df['comment_count'] == df['comment_count'].min()]

In [ ]:
# video dengan komentar tersedikit, meskipun comments_disabled false
df.loc[(df['comment_count'] == df['comment_count'].min()) & (df['comments_disabled'] == False)]

## Statistik data Publish Time

In [ ]:
# pengelompokan data berdasar hari di-publish
df["publish_time"].apply(lambda x: x.strftime('%A')).value_counts().to_frame().reset_index()


## Statistik data Trending Date

In [ ]:
# pengelompokan data berdasar bulan trending
trending = df.groupby([(df.trending_date.dt.year),(df.trending_date.dt.month)]).sum()
trending

# **Visualisasi Data**

### 10 Youtube channel teratas yang paling sering muncul di Trending Videos

In [ ]:
# horizontal bar plot dari 10 youtube channel teratas yang paling sering muncul di trending 
df['channel_title'].value_counts().head(10).sort_values().plot(kind='barh', figsize=(10,8))
plt.title("10 Youtube Channel teratas yang paling sering muncul di Trending Videos", size=(15))
plt.grid(alpha=0.4)
plt.show()

Data diatas menunjukkan channel yang videonya paling banyak masuk Trending, dengan channel nomor 1 adalah ESPN.

Berdasarkan grafik diatas, channel yang paling sering masuk Trending biasanya adalah channel yang kontennya rutin dan banyak penggemar, seperti basket dan reality show.

### Jumlah video berdasarkan kategori

In [ ]:
# horizontal bar plot dari jumlah video berdasarkan kategori
df['category_name'].value_counts().sort_values().plot(kind='barh', figsize=(10,8))
plt.title("Jumlah Video Berdasarkan Kategori", size=15)
plt.grid(alpha=0.4)
plt.show()

Grafik diatas adalah grafik jumlah video berdasarkan kategori, dengan kategori dengan jumlah video terbanyak adalah Entertainment.

Insight yang didapatkan dari grafik ini adalah kategori entertainment paling sering masuk trending, yang disusul dengan Music. Ini artinya banyak pengguna YouTube yang menonton untuk mendapatkan konten hiburan/mendengar musik.

### Perbandingan Jumlah Views Setiap Kategori

In [ ]:
# line plot perbandingan jumlah nilai views setiap kategori
df.groupby(["trending_date","category_name"])["views"].sum().unstack().plot(kind="line",figsize=(17,8),title="Perbandingan jumlah nilai views setiap kategori")
plt.xlabel("Tanggal")
plt.ylabel("Views")
plt.show()

Dari grafik diatas, diketahui bahwa kategori Music naik mendominasi Trending dimulai dari bulan April 2018. Sebelum April 2018, kategori Music dan Entertainment bersaing untuk mendapatkan views terbanyak pada video yang trending.

In [ ]:
# line plot perbandingan jumlah nilai views setiap kategori pada tahun 2017
df.loc[df["trending_date"].dt.year == 2017].groupby(["trending_date","category_name"])["views"].sum().unstack().plot(kind="line",figsize=(17,8),title="Perbandingan jumlah nilai views setiap kategori pada tahun 2017")
plt.xlabel("Tanggal")
plt.ylabel("Views")
plt.show()

Berbeda dengan keseluruhan data, pada tahun 2017 kategori Entertainment lebih mendominasi.

In [ ]:
# line plot perbandingan jumlah nilai views setiap kategori pada tahun 2018
df.loc[df["trending_date"].dt.year == 2018].groupby(["trending_date","category_name"])["views"].sum().unstack().plot(kind="line",figsize=(17,8),title="Perbandingan jumlah nilai views setiap kategori pada tahun 2018")
plt.xlabel("Tanggal")
plt.ylabel("Views")
plt.show()

### Perbandingan Jumlah Views Setiap Kategori

In [ ]:
# line plot perbandingan data tiap bulannya
trending.plot(kind="line", figsize = (17,8), title='Perbandingan data tiap bulan')

### Hubungan keseluruhan-bagian

In [ ]:
# pie chart
plt.figure(figsize=(10,10))
df.groupby('category_name').sum()['views'].plot(kind='pie', title='Perbandingan Jumlah Views Setiap Kategori', legend=True)

In [ ]:
# horizontal stacked bar chart Perbandingan Jumlah Views, Likes, dan Comments Setiap Kategori
df.groupby(['category_name']).sum().plot(kind='barh', y=['views', 'likes','dislikes', 'comment_count'], stacked=True)

In [ ]:
trending.plot(kind = "barh", stacked = True, y = ["views", "likes", "comment_count","dislikes"])

In [ ]:
trending.plot(kind = "barh", stacked = True, y = ["likes", "comment_count","dislikes"])

### Visualisasi korelasi views dan likes

In [ ]:
x = df["views"].values.reshape(-1,1)
y = df["likes"].values.reshape(-1,1)

regresi = LinearRegression().fit(x,y)
hasil = regresi.predict(x)
sns.scatterplot(x=df["views"], y=df["likes"])
plt.plot(x,hasil, color="red")
plt.xlabel("Views")
plt.ylabel("Likes")
plt.title("Plot Views vs Likes")
print("Koefisien korelasi antara views dan likes adalah " + str(df["views"].corr(df["likes"])))

Data diatas menunjukkan adanya korelasi positif antara views dan likes. Warna garis dibuat merah agar terlihat secara jelas garis regresi dari data.

Insight yang didapat adalah apabila views naik, maka likes juga akan cenderung naik.

In [ ]:
sns.jointplot(data=df, x="views", y="likes",hue = "category_name", kind="kde", figsize = (18,8))

Dapat dilihat semua berkumpul pada satu daerah

### Korelasi antar kolom data

In [ ]:
correlation_list = ['views', 'likes', 'dislikes', 'comment_count']
hm_data = df[correlation_list].corr() 
display(hm_data)

Tabel diatas merupakan tabel korelasi antar variabel kuantitatif. Korelasi terbesar ditunjukkan oleh like-view dengan nilai R 0.849.

Insight yang didapatkan dari tabel korelasi ini adalah apabila views tinggi, maka kecenderungan likes dan comment akan naik pula. Dan apabila likes atau dislikes tinggi, kecenderungan untuk mendapatkan comment naik juga. 

Yang dapat disimpulkan adalah, video trending biasanya viral dengan banyak likes yang berbanding lurus dengan views.

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(hm_data, annot=True);

Diatas adalah heatmap dari data korelasi yang telah dipaparkan diatas, agar lebih mudah dilihat dalam bentuk visual.

In [ ]:
scatter_matrix(df[correlation_list], alpha=0.2, figsize=(6, 6), diagonal='kde')